In [5]:
import os
import shutil
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm 

# Function to apply the grayscale mask to the original image
def apply_mask_to_image(image_path, mask_path, output_path):
    # Read the original image and mask
    original_image = cv2.imread(image_path)
    grayscale_mask = np.load(mask_path)
    
    # Ensure the mask has the same dimensions as the image
    h, w, _ = original_image.shape
    grayscale_mask = cv2.resize(grayscale_mask, (w, h))
    
    # Convert the mask to 3D
    grayscale_mask_3d = np.repeat(grayscale_mask[:, :, np.newaxis], 3, axis=2)
    
    # Apply the mask to the original image
    masked_image = original_image * grayscale_mask_3d
    
    # Save the masked image
    cv2.imwrite(output_path, masked_image)

# # Main code starts here
# base_path = "/home/workstation/code/XAImethods/hf_cam_dev/results/apple/mobilevit-small/GradCAM"

# # Allow specifying the location of the masking folder
# masking_root_folder = "/home/workstation/code/XAImethods/hf_cam_dev/evaluation/apple/mobilevit-small/GradCAM"

# Define the common root directory
root_directory = "/home/workstation/code/XAImethods/hf_cam_dev"

# Take the user-specified subdirectory
user_subdirectory = "results/apple/mobilevit-small/HiResCAM"  # This is what the user would change as needed

# Construct the base path and the masking root folder from the root and user subdirectory
base_path = os.path.join(root_directory, user_subdirectory)
masking_subpath = user_subdirectory.replace("results", "evaluation", 1)
masking_root_folder = os.path.join(root_directory, masking_subpath)


if not os.path.exists(masking_root_folder):
    os.makedirs(masking_root_folder)

# Using tqdm to wrap around the os.listdir() to show progress
for subfolder in tqdm(os.listdir(base_path), desc="Processing folders"):
    subfolder_path = os.path.join(base_path, subfolder)
    
    if os.path.isdir(subfolder_path):
        output_folder = os.path.join(masking_root_folder, subfolder)
        
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        for filename in os.listdir(subfolder_path):
            src_path = os.path.join(subfolder_path, filename)
            dst_path = os.path.join(output_folder, filename)
            
            if os.path.isfile(src_path):
                if filename == "original.jpg":
                    mask_path = os.path.join(subfolder_path, "grayscale.npy")
                    if os.path.exists(mask_path):
                        apply_mask_to_image(src_path, mask_path, os.path.join(output_folder, "masked_image.jpg"))
                    shutil.copy(src_path, dst_path)  # 这一行会复制 original.jpg 到目标文件夹
                else:
                    shutil.copy(src_path, dst_path)

print("Masking and file copying completed!")


Processing folders: 100%|██████████| 3896/3896 [00:47<00:00, 81.37it/s]

Masking and file copying completed!
